## Modulation Recognition Example: RML2016.10a Dataset + VT-CNN2 Mod-Rec Network

This work is copyright DeepSig Inc. 2017.
It is provided open source under the Create Commons Attribution-NonCommercial 4.0 International (CC BY-NC 4.0) Licence
https://creativecommons.org/licenses/by-nc/4.0/

Use of this work, or derivitives inspired by this work is permitted for non-commercial usage only and with explicit citaiton of this original work.

A more detailed description of this work can be found at
https://arxiv.org/abs/1602.04105

A more detailed description of the RML2016.10a dataset can be found at
http://pubs.gnuradio.org/index.php/grcon/article/view/11

Citation of this work is required in derivative works:

```
@article{convnetmodrec,
  title={Convolutional Radio Modulation Recognition Networks},
  author={O'Shea, Timothy J and Corgan, Johnathan and Clancy, T. Charles},
  journal={arXiv preprint arXiv:1602.04105},
  year={2016}
}
@article{rml_datasets,
  title={Radio Machine Learning Dataset Generation with GNU Radio},
  author={O'Shea, Timothy J and West, Nathan},
  journal={Proceedings of the 6th GNU Radio Conference},
  year={2016}
}
```

The RML2016.10a dataset is used for this work (https://radioml.com/datasets/)


In [1]:
import os, random
import numpy as np
import seaborn as sns
import sys, keras, pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import theano as th
import theano.tensor as T
# setting env variables before Keras import you can set up which backend and which GPU it uses
os.environ["KERAS_BACKEND"] = "theano"
#os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["THEANO_FLAGS"] = "device=cpu%d" % (1)

In [3]:
from keras.utils import np_utils
import keras.models as models
from keras.layers.core import Reshape, Dense, Dropout, Activation, Flatten
from keras.layers.noise import GaussianNoise
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.regularizers import *
from keras.optimizers import Adam

# Dataset setup

In [4]:
# Load dataset ...
# You will need to seperately download or generate this file
import pprint
import sys

# pickle.load?

In [5]:
pkl_file = open('./RML2016.10a/RML2016.10a_dict.pkl', 'rb')
Xd = pickle.load(pkl_file, encoding='bytes')
len(Xd)

220

In [6]:
for k, v in Xd.items():
    print(k)
    print(len(v))
    break

(b'QPSK', 2)
1000


In [8]:
print(len(key), key[0][0])

220 b'QAM16'


In [9]:
modulation_type = set()
for i in range(len(key)):
    modulation_type.add(key[i][0])

modulation_type = list(modulation_type)
print(len(modulation_type), modulation_type)

11 [b'WBFM', b'AM-DSB', b'PAM4', b'QAM16', b'BPSK', b'AM-SSB', b'CPFSK', b'GFSK', b'8PSK', b'QAM64', b'QPSK']


In [12]:
snrs, mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))),
                 [1, 0])
print(len(snrs), len(mods))
print(snrs, mods)

20 11
[-20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18] [b'8PSK', b'AM-DSB', b'AM-SSB', b'BPSK', b'CPFSK', b'GFSK', b'PAM4', b'QAM16', b'QAM64', b'QPSK', b'WBFM']


In [13]:
X = []
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod, snr)])
        for i in range(Xd[(mod, snr)].shape[0]):
            lbl.append((mod, snr))
X = np.vstack(X)

In [14]:
print(len(X[0][0]), len(X[0][1]), X.shape)
print(len(lbl))
lbl[:5]

128 128 (220000, 2, 128)
220000


[(b'8PSK', -20),
 (b'8PSK', -20),
 (b'8PSK', -20),
 (b'8PSK', -20),
 (b'8PSK', -20)]

In [15]:
def to_onehot(yy):
    yy1 = np.zeros([len(yy), max(yy) + 1])
    yy1[np.arange(len(yy)), yy] = 1
    return yy1


to_onehot([8, 9, 10])

array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [ ]:
# Partition data into training and test sets
# keeping SNR and Mod labels handy for each

np.random.seed(2020)
n_examples = X.shape[0]
n_train = int(np.floor(n_examples * 0.8))
print(n_train)

train_idx = np.random.choice(range(0, n_examples), size=n_train, replace=False)
test_idx = list(set(range(0, n_examples)) - set(train_idx))

X_train = X[train_idx]
X_test = X[test_idx]
print(len(X_train[0][0]))

y_train = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), train_idx)))
y_test = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), test_idx)))

In [23]:
print(lbl[100000][0], mods.index(lbl[100000][0]))

b'GFSK' 5


In [25]:
print(list(map(lambda x: mods.index(lbl[x][0]), [0, 23000]))[:5])

[0, 1]


In [27]:
from sklearn.model_selection import train_test_split

y = np.array(
    to_onehot(list(map(lambda x: mods.index(lbl[x][0]), np.arange(len(lbl))))))
print(y[:5])

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)
len(X_train[0][0])

128

In [29]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(176000, 2, 128) (44000, 2, 128) (176000, 11) (44000, 11)


In [30]:
classes = mods
classes

[b'8PSK',
 b'AM-DSB',
 b'AM-SSB',
 b'BPSK',
 b'CPFSK',
 b'GFSK',
 b'PAM4',
 b'QAM16',
 b'QAM64',
 b'QPSK',
 b'WBFM']

# Build the NN Model

Build VT-CNN2 Neural Net model using Keras 

- Reshape [N,2,128] to [N,1,2,128] on input

- Pass through 2 2DConv/ReLu layers

- Pass through 2 Dense layers (ReLu and Softmax)

- Perform categorical cross entropy optimization

In [31]:
in_shp = list(X_train.shape[1:])
in_shp

[2, 128]

In [ ]:
# ZeroPadding2D?

In [33]:
model = models.Sequential()

# dropout rate
DROP = 0.5
model = models.Sequential()

# reshape input
model.add(Reshape([1] + in_shp, input_shape=in_shp))

# zeropadding2d_1
model.add(ZeroPadding2D((0, 2), data_format="channels_first"))

# conv1
model.add(
    Convolution2D(
        filters=256,
        kernel_size=(1, 3),
        data_format="channels_first",
        activation="relu",
        name="conv1",
    ))

# dropout_1
model.add(Dropout(DROP))

# zeropadding2d_2
model.add(ZeroPadding2D((0, 2), data_format="channels_first"))

# conv2
model.add(
    Convolution2D(
        filters=80,
        kernel_size=(2, 3),
        data_format="channels_first",
        activation="relu",
        name="conv2",
    ))

# dropout_2
model.add(Dropout(DROP))

# flatten_1
model.add(Flatten())

# dense1
model.add(Dense(256, activation='relu', name="dense1"))

# dropout_3
model.add(Dropout(DROP))

# dense2
model.add(Dense(len(classes), name="dense2"))

# activation_1
model.add(Activation('softmax'))

# reshape_2
model.add(Reshape([len(classes)]))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 1, 2, 128)         0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 1, 2, 132)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 256, 2, 130)       1024      
_________________________________________________________________
dropout_2 (Dropout)          (None, 256, 2, 130)       0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 256, 2, 134)       0         
_________________________________________________________________
conv2 (Conv2D)               (None, 80, 1, 132)        122960    
_________________________________________________________________
dropout_3 (Dropout)          (None, 80, 1, 132)       

In [34]:
# number of epochs to train
# EPOCH = 100
EPOCH = 1

# training batch size
BATCH = 1024

# Train the Model

In [ ]:
# model.fit?

In [35]:
# perform training ...
# call the main training loop in keras

filepath = 'convmodrecnets_CNN2_0.5.wts.h5'

history = model.fit(X_train,
                    y_train,
                    batch_size=BATCH,
                    epochs=EPOCH,
                    verbose=2,
                    validation_data=(X_test, y_test),
                    callbacks=[
                        keras.callbacks.ModelCheckpoint(filepath,
                                                        monitor='val_loss',
                                                        verbose=0,
                                                        save_best_only=True,
                                                        mode='auto'),
                        keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=5,
                                                      verbose=0,
                                                      mode='auto')
                    ])

# we re-load the best weights once training is finished
model.load_weights(filepath)

InvalidArgumentError:  Conv2DCustomBackpropInputOp only supports NHWC.
	 [[node gradient_tape/sequential_3/conv2/Conv2D/Conv2DBackpropInput (defined at <ipython-input-35-661a7782236e>:6) ]] [Op:__inference_train_function_1201]

Function call stack:
train_function


# Evaluate and Plot Model Performance

In [ ]:
# Show simple version of performance

score = model.evaluate(X_test,
                       y_test,
                       show_accuracy=True,
                       verbose=0,
                       batch_size=BATCH)
print(score)

In [ ]:
# Show loss curves
plt.figure()
plt.title('Training performance')
plt.plot(history.epoch, history.history['loss'], label='train loss+error')
plt.plot(history.epoch, history.history['val_loss'], label='val_error')
plt.legend()

In [ ]:
def plot_confusion_matrix(cm,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues,
                          labels=[]):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Plot confusion matrix
test_y_hat = model.predict(X_test, batch_size=BATCH)
conf = np.zeros([len(classes), len(classes)])
confnorm = np.zeros([len(classes), len(classes)])

for i in range(0, X_test.shape[0]):
    j = list(y_test[i, :]).index(1)
    k = int(np.argmax(test_y_hat[i, :]))
    conf[j, k] = conf[j, k] + 1

for i in range(0, len(classes)):
    confnorm[i, :] = conf[i, :] / np.sum(conf[i, :])

plot_confusion_matrix(confnorm, labels=classes)

In [ ]:
# Plot confusion matrix
acc = {}
for snr in snrs:

    # extract classes @ SNR
    test_SNRs = map(lambda x: lbl[x][1], test_idx)
    test_X_i = X_test[np.where(np.array(test_SNRs) == snr)]
    test_y_i = y_test[np.where(np.array(test_SNRs) == snr)]

    # estimate classes
    test_y_i_hat = model.predict(test_X_i)
    conf = np.zeros([len(classes), len(classes)])
    confnorm = np.zeros([len(classes), len(classes)])
    for i in range(0, test_X_i.shape[0]):
        j = list(test_Y_i[i, :]).index(1)
        k = int(np.argmax(test_Y_i_hat[i, :]))
        conf[j, k] = conf[j, k] + 1
    for i in range(0, len(classes)):
        confnorm[i, :] = conf[i, :] / np.sum(conf[i, :])
    plt.figure()
    plot_confusion_matrix(confnorm,
                          labels=classes,
                          title="ConvNet Confusion Matrix (SNR=%d)" % (snr))

    cor = np.sum(np.diag(conf))
    ncor = np.sum(conf) - cor
    print "Overall Accuracy: ", cor / (cor + ncor)
    acc[snr] = 1.0 * cor / (cor + ncor)

In [ ]:
# Save results to a pickle file for plotting later

print(acc)
fd = open('results_cnn2_d0.5.dat', 'wb')
cPickle.dump(("CNN2", 0.5, acc), fd)

In [ ]:
# Plot accuracy curve
plt.plot(snrs, map(lambda x: acc[x], snrs))
plt.xlabel("Signal to Noise Ratio")
plt.ylabel("Classification Accuracy")
plt.title("CNN2 Classification Accuracy on RadioML 2016.10 Alpha")